In [ ]:
import sys
import os

In [ ]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv

load_dotenv()

In [ ]:
os.environ['BUILD_PROJECT_PATH'] = "/Users/jamesalner/fine-tuning-build-project"

In [ ]:
# Either hard code your path to the project, or set it as an environment variable
build_project_path = os.environ['BUILD_PROJECT_PATH']

In [ ]:
synthetic_data_path = os.path.join(build_project_path, 'synthetic_data')
data_path = os.path.join(synthetic_data_path, 'data')

In [ ]:
if synthetic_data_path not in sys.path:
    sys.path.append(synthetic_data_path)

In [ ]:
from llm_requests import generate_prompt

seed_titles_df = pd.read_csv(os.path.join(data_path, 'seed_titles.csv'))

seed_titles = seed_titles_df['seed_title'].unique()

example_seed_titles = np.random.choice(seed_titles, 1)

example_prompt = generate_prompt(example_seed_titles, num_examples_per_title=15)

<h3>Experiment with different models and see how they compare!</h3>

In [ ]:
# model_name = 'Meta-Llama-3.3-70B-Instruct'
# model_name = 'Meta-Llama-3.1-405B-Instruct'
# model_name = 'gemini-2.0-flash'
model_name = 'gemma-3-27b-it'

In [ ]:
from llm_requests import get_client

client = get_client(api_key_name='GEMINI_API_KEY', base_url="https://generativelanguage.googleapis.com/v1beta/openai/")


In [ ]:
from llm_requests import async_make_api_call

example_response, _ = await async_make_api_call(client, model_name, example_prompt, perturbation_std=0.1)

In [ ]:
example_response.choices[0].message.content

In [ ]:
from gpt_parsing import parse_gpt_response

parsed_output = parse_gpt_response(example_response.choices[0].message.content,1,15)

In [ ]:
for seed_title, response in zip(example_seed_titles, parsed_output):
    print(f'Variations of: {seed_title}:')
    print('-------------------')
    for i, variation in enumerate(response):
        print(f'{i+1}: {variation}')

    print('\n\n')

In [ ]:
from llm_requests import async_main_stubborn

output_dict_path = os.path.join(data_path, 'jitter_responses.pkl')
response_dict = await async_main_stubborn(
    all_query_titles=seed_titles,
    client=client,
    model_name=model_name,
    output_path=output_dict_path,
    chunk_size=1,
    num_examples_per_title=10,
    giveup_after=10,  # More attempts
    delay=2.1,  # Slightly longer delay
    initial_backoff=10,  # Start with 10 seconds backoff
    max_backoff=300  # Maximum 5 minutes backoff for severe rate limiting
)

In [ ]:
import pickle

output_dict_path = os.path.join(data_path, 'jitter_responses.pkl')

with open(output_dict_path, 'rb') as f:
    response_dict = pickle.load(f)

In [ ]:
jitter_df = {
    'jittered_title': [],
    'seed_title': [],
}

for seed_title, jittered_titles in response_dict.items():
    for jittered_title in jittered_titles:
        jitter_df['jittered_title'].append(jittered_title)
        jitter_df['seed_title'].append(seed_title)

jitter_df = pd.DataFrame(jitter_df)

jitter_df = jitter_df.merge(seed_titles_df, on='seed_title', how='left')
jitter_df.sample(5)

In [ ]:
jitter_df.to_csv(os.path.join(data_path, 'jittered_titles.csv'), index=False)